In [ ]:
!pip install -U huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2" #"google/medgemma-27b-text-it"  # ✅ This must be a string
local_dir = "./my_local_model"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.save_pretrained(local_dir)
model.save_pretrained(local_dir)

print(f"Model and tokenizer saved successfully to: {local_dir}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model and tokenizer saved successfully to: ./my_local_model


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "input": [
        "What is the capital of France?",
        "Who wrote 'Pride and Prejudice'?"
    ],
    "output": [
        "The capital of France is Paris.",
        "Jane Austen wrote 'Pride and Prejudice'."
    ]
})


In [ ]:
df.to_json("./finetune_data.json", orient="records", lines=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./my_local_model"  # directory containing downloaded model files

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


In [ ]:
import pandas as pd

json_path = "./finetune_data.json"
df = pd.read_json(json_path, lines=True)
print(df.head())

from datasets import Dataset
dataset = Dataset.from_pandas(df)


                              input                                    output
0    What is the capital of France?           The capital of France is Paris.
1  Who wrote 'Pride and Prejudice'?  Jane Austen wrote 'Pride and Prejudice'.


In [ ]:
def tokenize_fn(example):
    prompt = example["input"]
    response = example["output"]
    text = f"### Question: {prompt}\n### Answer: {response}"

    # Tokenize with padding/truncation
    encoding = tokenizer(text, truncation=True, max_length=512, padding="max_length")

    # For causal LM, labels are input_ids with padding tokens replaced by -100 (ignored by loss)
    labels = encoding["input_ids"].copy()
    labels = [l if l != tokenizer.pad_token_id else -100 for l in labels]

    encoding["labels"] = labels
    return encoding

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = dataset.map(tokenize_fn, batched=False)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=5e-5,
    save_total_limit=1,
    fp16=True,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-10-4001563720.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=2, training_loss=2.7177295684814453, metrics={'train_runtime': 61.73, 'train_samples_per_second': 0.065, 'train_steps_per_second': 0.032, 'total_flos': 1045168128000.0, 'train_loss': 2.7177295684814453, 'epoch': 2.0})

In [ ]:
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.json',
 './finetuned_model/merges.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!apt update
!apt install -y build-essential cmake
!rm -rf llama.cpp/build
!cmake -S llama.cpp -B llama.cpp/build
!cmake --build llama.cpp/build --config Release

Cloning into 'llama.cpp'...
remote: Enumerating objects: 55800, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 55800 (delta 53), reused 27 (delta 27), pack-reused 55713 (from 2)
Receiving objects: 100% (55800/55800), 132.99 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (40713/40713), done.


In [ ]:
!cd llama.cpp && python3 convert_hf_to_gguf.py --outfile /content/finetuned_model.gguf --outtype q8_0 /content/finetuned_model


INFO:hf-to-gguf:Loading model: finetuned_model
INFO:hf-to-gguf:Model architecture: GPT2LMHeadModel
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:blk.0.attn_qkv.bias,       torch.float32 --> F32, shape = {2304}
INFO:hf-to-gguf:blk.0.attn_qkv.weight,     torch.float32 --> Q8_0, shape = {768, 2304}
INFO:hf-to-gguf:blk.0.attn_output.bias,    torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.attn_output.weight,  torch.float32 --> Q8_0, shape = {768, 768}
INFO:hf-to-gguf:blk.0.attn_norm.bias,      torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_norm.bias,       torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float32 --> F32, shape = {768}
INFO:hf-to-gguf:blk.0.ffn_up.bias,         torch.float32 --> F32, shape = {3072}
I

In [ ]:
!pip install llama-cpp-python
#--force-reinstall --upgrade --no-cache-dir llama-cpp-python --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.12-cp311-cp311-linux_x86_64.whl size=4158972 sha256=9a8551716c2733f5d5cfe2d7b5a8a9813bec65305d95fc1ce5d635c44e52e597
  Stored in directory: /root/.cache/pip/wheels/7d/b8/30/8988efe892bf4e389f95fa946371363596b28a47091f12494c
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

# Load your model (path to the .gguf file)
llm = Llama(
    model_path="./finetuned_model.gguf",
    n_ctx=2048,
    n_gpu_layers=20  # depends on your GPU
)

# Run inference
response = llm("Question: What is the capital of Tunisia? \nAnswer: ")
print(response['choices'][0]['text'])


llama_model_loader: loaded meta data with 21 key-value pairs and 148 tensors from ./finetuned_model.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gpt2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Finetuned_Model
llama_model_loader: - kv   3:                         general.size_label str              = 124M
llama_model_loader: - kv   4:                           gpt2.block_count u32              = 12
llama_model_loader: - kv   5:                        gpt2.context_length u32              = 1024
llama_model_loader: - kv   6:                      gpt2.embedding_length u32              = 768
llama_model_loader: - kv   7:                   gpt2.feed_forward_length u32      


The capital of Tunisia is Tuz Khartoum.  The
